### Spread Option Pricer
#### An option that expresses a view on spread between two stocks, if one stock is expected to out perform another stock, regardless of direction of overall market 
* Write a Spread option class that takes following parameters
    * underlying names (Name1, Name2), price multiplier for the two stocks (M1, M2), spread strike (K), time to expiry(T), option type (Call or Put) 
* Write a simulation pricer to price spread option. This function takes following parameters
    * Object of type SpreadOption
    * current spot prices for the two stocks (S1, S2)
    * volatility for two stocks (sigma1, sigma2)
    * Interest rate (rf)
    * Correlation between two stocks (rho)
* Pay off function is Max((ST1 * M1 - ST2 * M2) -K, 0) for Call option and MAX of (K - (ST1*M1 - ST2*M2), 0) for Put option
* Write a script to price following Spread Option under 10000 paths
    * Spread option between Apple and Google
    * Option type Call
    * Multiplier 5, 1
    * Strike 0
    * T = 0.5 (Six months)
    * Spot prices (Apple = 250, Google = 1250)
    * Volatility (Apple 30%, Google 30%)
    * Risk Free rate 3%
    * Correlation 75%
   
* Hint: see function numpy.random.multivariate_normal to generate correlated normal random numbers

In [9]:
# imports
import numpy as np
from scipy import optimize
from scipy.stats import norm
import math
import matplotlib.pyplot as plt



In [16]:
class SpreadOption:
    def __init__(self, name1, name2, M1, M2, K, T, optiontype):
        self.name1 = name1
        self.name2 = name2
        self.M1 = M1
        self.M2 = M2
        self.K = K
        self.T = T
        self.optiontype = optiontype
        
    def price(self, stock1spotprice, stock2spotprice, stock1volatility, stock2volatility, r ,correlation):
        dt= (self.T/100)
        stock1sqrtdt= stock1volatility*math.sqrt(self.T/100)
        stock2sqrtdt=stock2volatility*math.sqrt(self.T/100)
        stock1drift= (r-(0.5*stock1volatility*stock1volatility))*math.sqrt(self.T/100)
        stock2drift= (r-(0.5*stock1volatility*stock2volatility))*math.sqrt(self.T/100)
        stock1table = np.zeros([101, 1000])
        stock2table = np.zeros([101, 1000])

        for i in range(0,100):
            for z in range(0,1000):
                mean = (0,0)
                cov = [[1.0, correlation],[correlation, 1.0]]
                stock1table[0,:]=stock1spotprice
                stock2table[0,:]=stock2spotprice
                x,y = np.random.multivariate_normal(mean,cov,(10000)).T
                stock1table[i+1,z] = stock1table[i,z]*np.exp(stock1drift+stock1sqrtdt*x[z])
                stock2table[i+1,z] = stock2table[i,z]*np.exp(stock1drift+stock2sqrtdt*y[z])
                
        stock1change = stock1spotprice*np.exp(r*self.T)-np.mean(stock1table[-1,:])
        stock2change = stock2spotprice*np.exp(r*self.T)-np.mean(stock2table[-1,:])
        adjustedstock1 = list(map(lambda x: x + stock1change,stock1table[-1,:]))
        adjustedstock2 = list(map(lambda x: x + stock1change,stock2table[-1,:]))

        np.mean(adjustedstock1)
        np.mean(adjustedstock2)

        if self.optiontype == "C":
            self.optiontype = 1
            expectedpayoff = list(map(lambda x: max(self.optiontype*(np.mean(x)*self.M1 - np.mean(adjustedstock2)* self.M2) -self.K, 0),adjustedstock1))
            expectedpayoffPV= np.mean(expectedpayoff)*np.exp(-r*self.T)
            return("The PV of the option is ",expectedpayoffPV)
        else:
            self.optiontype = -1
            expectedpayoff = list(map(lambda x: max(self.optiontype*(np.mean(x)*self.M1 - np.mean(adjustedstock2)* self.M2) -self.K, 0),adjustedstock1))

            expectedpayoffPV= np.mean(expectedpayoff)*np.exp(-r*self.T)

            return("The PV of the option is ",expectedpayoffPV)





In [17]:
a1 = SpreadOption("Apple", "Google",5,1,0, 0.5, "C" )

In [18]:
a1.price(250,1250, 0.3, 0.3, 0.03,0.75)

('The PV of the option is ', 144.21029462505786)